<a href="https://colab.research.google.com/github/Gregory-Eales/LANL-Earthquake-Prediction/blob/master/LANL_Earthquake_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LANL Earthquake Prediction on Colab

## To Do:
- Create Data  Loading Functions
- Create Grid Search Functions: Dimensions (model, parameters,  data input size)
- 

## Setting up

### Import Dependencies

In [0]:
# import dependencies
import numpy as np
import pandas as pd
import tensorflow as tf

### Mount Drive for  Cleaned Data Access

In [4]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Define Data Loading Functions

In [0]:
# set data paths from drive
training_data_path =  "drive/My Drive/LANL-Data-Colab/LANL-Training-Data"
testing_data_path = "drive/My Drive/LANL-Data-Colab/LANL-Test-Data"


def get_file_number(n):
    n = str(n)
    n = "0"*(3-len(n)) + n
    return n

def load_data(number):
    path = "drive/My Drive/LANL-Data-Colab/LANL-Training-Data/Training-Data"
    path = path + get_file_number(number) + ".csv"
    train_df = pd.read_csv(path, dtype={'acoustic_data': np.int8, 'time_to_failure': np.float32})
    return train_df

def create_training_data(number, data_length):
    train_df = load_data(number)
    samples = train_df.shape[0]//data_length
    X = train_df["acoustic_data"].as_matrix()
    Y = train_df["time_to_failure"].as_matrix()
    del train_df
    train_df = None
    x_train = []
    y_train = []
    for i in range(samples):
        x_train.append(X[i*data_length:(i+1)*data_length].tolist())
        y_train.append(np.sum(Y[i*data_length:(i+1)*data_length].tolist()))
    x_train.append(X[-1*data_length -1:-1].tolist())
    y_train.append(np.sum(Y[-1*data_length-1:-1]))
    print(type(x_train), len(x_train[-1]))
    x_train = np.reshape(np.array(x_train), [samples+1, data_length, 1])
    y_train = np.reshape(np.array(y_train), [samples+1, 1])
    y_train = y_train/data_length
    return x_train, y_train

In [32]:
x, y = create_training_data(1, 10000)

<class 'list'> 10000


## Model Evaluation

### LSTM Neural Networks

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, RNN

Using TensorFlow backend.


In [0]:
# define LSTM Models for training

# LSTM 01
LSTM_Model_01 = Sequential()
LSTM_Model_01.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_01.add(LSTM(15))
LSTM_Model_01.add(Dense(15, activation="relu"))
LSTM_Model_01.add(Dense(5, activation="relu"))
LSTM_Model_01.add(Dense(1, activation="relu"))
LSTM_Model_01.compile(optimizer="adam", loss='mean_squared_error')

# LSTM 02
LSTM_Model_02 = Sequential()
LSTM_Model_02.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_02.add(LSTM(15))
LSTM_Model_02.add(Dense(15, activation="relu"))
LSTM_Model_02.add(Dense(1, activation="relu"))
LSTM_Model_02.compile(optimizer="adam", loss='mean_squared_error')

# LSTM 03
LSTM_Model_03 = Sequential()
LSTM_Model_03.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_03.add(LSTM(15, return_sequences=True))
LSTM_Model_03.add(LSTM(15))
LSTM_Model_03.add(Dense(1, activation="relu"))
LSTM_Model_03.compile(optimizer="adam", loss='mean_squared_error')

# LSTM 04
LSTM_Model_04 = Sequential()
LSTM_Model_04.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_04.add(LSTM(15, return_sequences=True))
LSTM_Model_04.add(LSTM(15, return_sequences=True))
LSTM_Model_04.add(LSTM(15))
LSTM_Model_04.add(Dense(1, activation="relu"))
LSTM_Model_04.compile(optimizer="adam", loss='mean_squared_error')


LSTM_Models = [LSTM_Model_01, LSTM_Model_02, LSTM_Model_03, LSTM_Model_04]

In [0]:
def train_LSTM_Models(LSTM_Models):
  
  for file_num in range(1):
    x_train, y_train = create_training_data(file_num, 10000)
    for number, lstm_model in enumerate(LSTM_Models):
      lstm_model.load_weights("LSTM_Model_0" + str(number+1) + ".h5")
      lstm_model.fit(x_train[0:1], y_train[0:1])
      lstm_model.save_weights("LSTM_Model_0" + str(number+1) + ".h5")

In [15]:
train_LSTM_Models(LSTM_Models)

<class 'list'> 10000
Epoch 1/1
1/1 [==============================] - 31s 31s/step - loss: 1.6319
Epoch 1/1
1/1 [==============================] - 31s 31s/step - loss: 2.1559
Epoch 1/1
1/1 [==============================] - 46s 46s/step - loss: 1.4090
Epoch 1/1
1/1 [==============================] - 62s 62s/step - loss: 1.8885


In [16]:
!ls

drive		  LSTM_Model_03.h5  LSTM_Model2.h5  sample_data
LSTM_Model_01.h5  LSTM_Model_04.h5  LSTM_Model3.h5
LSTM_Model_02.h5  LSTM_Model1.h5    LSTM_Model4.h5


### Support Vector Machines

### Neural Ordinary Differential Equations

## Training Final Model